Here I tried to add Facemask to my dataset using a facemask png but soon i realized that the facemask was being added to random positions instead of the face

In [11]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# Load mask image (make sure it has transparency)
mask_image = cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)  # Load with alpha channel

# Function to add mask to a face
def add_mask_to_face(face_img, mask_img):
    # Convert face image to RGBA if not already
    if face_img.shape[2] != 4:
        face_img = cv2.cvtColor(face_img, cv2.COLOR_RGB2RGBA)
    
    # Resize mask to fit the face width
    mask_resized = cv2.resize(mask_img, (face_img.shape[1], int(face_img.shape[0] / 2)))
    
    # Determine the region to overlay the mask
    y1, y2 = face_img.shape[0] // 2, face_img.shape[0] // 2 + mask_resized.shape[0]
    x1, x2 = 0, face_img.shape[1]
    
    # Overlay the mask on the face
    alpha_mask = mask_resized[:, :, 3] / 255.0
    alpha_face = 1.0 - alpha_mask

    for c in range(0, 3):
        face_img[y1:y2, x1:x2, c] = (alpha_mask * mask_resized[:, :, c] + alpha_face * face_img[y1:y2, x1:x2, c])
    
    return face_img

# Directory paths
dataset_dir = r"C:\Users\DELL\Downloads\part1\part1"
masked_dataset_dir = r"C:\Users\DELL\Downloads\part2\part2"

# Create output directory if it doesn't exist
if not os.path.exists(masked_dataset_dir):
    os.makedirs(masked_dataset_dir)

# Process images in the dataset directory
for subdir, dirs, files in os.walk(dataset_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        image = cv2.imread(file_path)
        
        if image is not None:
           face_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Add mask to the face
           masked_face = add_mask_to_face(face_image, mask_image)

    # Save the masked face image
           output_path = os.path.join(masked_dataset_dir, file)
           cv2.imwrite(output_path, cv2.cvtColor(masked_face, cv2.COLOR_RGBA2BGR))
        else:
            print(f"Failed to read image: {file_path}")

Failed to read image: C:\Users\DELL\Downloads\part1\part1\.DS_Store


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:696: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'


I did a bit of reseach and came across a face detection model called Haarcascade. Here I used haarcascade for face detection and then i tried to put the mask on but the results were still not satisfacotry and the mask was getting put on in a weird and random way 

In [13]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the mask image
mask_image = cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)

# Function to add mask to a face
def add_mask_to_face(image, mask_img, face_position):
    x, y, w, h = face_position
    
    # Resize mask to fit the face width
    mask_resized = cv2.resize(mask_img, (w, h))
    
    # Determine the region to overlay the mask
    y1, y2 = max(0, y), min(y + mask_resized.shape[0], image.shape[0])
    x1, x2 = max(0, x), min(x + mask_resized.shape[1], image.shape[1])

    mask_y1, mask_y2 = 0, y2 - y1
    mask_x1, mask_x2 = 0, x2 - x1

    # Only apply the mask if dimensions are correct
    if y2 > y1 and x2 > x1:
        alpha_mask = mask_resized[mask_y1:mask_y2, mask_x1:mask_x2, 3] / 255.0
        alpha_face = 1.0 - alpha_mask

        for c in range(0, 3):
            image[y1:y2, x1:x2, c] = (alpha_mask * mask_resized[mask_y1:mask_y2, mask_x1:mask_x2, c] + 
                                      alpha_face * image[y1:y2, x1:x2, c])

    return image

# Directory paths
dataset_dir = r"C:\Users\DELL\Downloads\part1\part1"
masked_dataset_dir = r"C:\Users\DELL\Downloads\masked_dataset"

# Create output directory if it doesn't exist
if not os.path.exists(masked_dataset_dir):
    os.makedirs(masked_dataset_dir)

# Process images in the dataset directory
for subdir, dirs, files in os.walk(dataset_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        image = cv2.imread(file_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate over detected faces and add mask
        for (x, y, w, h) in faces:
            face_position = (x, y, w, h)

            # Add mask to the face
            masked_image = add_mask_to_face(image, mask_image, face_position)

        # Save the image with masked faces
        output_path = os.path.join(masked_dataset_dir, file)
        cv2.imwrite(output_path, masked_image)


KeyboardInterrupt: 

Up unitl this part my approach was to add masks and then train the model but that wasn't working and keeping in mind that the masks could be of different coulours in our testing data it didn't seem accurate. Here I decided to take a turn to a different approach. Now I am using my "Original Dataset" to make another dataset in which we are detecting the faces using haar cascade and then cropping them out . The results were satisfactory but still some images did not contain a face at all which takes us to the next part.

In [39]:
import cv2
import os
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
dataset_dir = r"C:\Users\DELL\Downloads\ikrima faces\Original Dataset"
cropped_faces_dir = r"C:\Users\DELL\Downloads\ikrima faces\Face_detected_dataset"

if not os.path.exists(cropped_faces_dir):
    os.makedirs(cropped_faces_dir)
for subdir, dirs, files in os.walk(dataset_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        image = cv2.imread(file_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(96, 96))
        for i, (x, y, w, h) in enumerate(faces):
            cropped_face = image[y:y+h, x:x+w]
            output_path = os.path.join(cropped_faces_dir, f"{os.path.splitext(file)[0]}_face{i}.jpg")
            cv2.imwrite(output_path, cropped_face)


In this part I again used haar cascade to detect face in our newly cropped image dataset i.e. "Face_detected_dataset" and whichever of them didnt have a face were deleted.

In [41]:
# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cropped_faces_dir = r"C:\Users\DELL\Downloads\ikrima faces\Face_detected_dataset"
for file in os.listdir(cropped_faces_dir):
    file_path = os.path.join(cropped_faces_dir, file)
    image = cv2.imread(file_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        os.remove(file_path)
        print(f"Deleted {file_path} - No face detected")


Deleted C:\Users\DELL\Downloads\ikrima faces\part2\105_0_0_20170112213001988_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\105_1_0_20170112213507183_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\105_1_1_20170112213303693_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110220033115_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110220111082_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110220644705_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110224238891_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110225227587_face0.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\10_0_0_20170110225451638_face1.jpg - No face detected
Deleted C:\Users\DELL\Downloads\ikrima faces\part2\1

Now in this part I deduced that a mask only usually covers 40 percent of the face so I cropped the bottom 40 percent and saved it in a folder


In [42]:

image_dir = r"C:\Users\DELL\Downloads\ikrima faces\Face_detected_dataset"
output_dir = r"C:\Users\DELL\Downloads\ikrima faces\Cropped_face_dataset"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for file in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file)
    image = cv2.imread(file_path)
    
    if image is None:
        print(f"Failed to load image {file_path}")
        continue
    height, width, _ = image.shape
 
    crop_height = int(height * 0.4)

    roi = image[:-crop_height, :]

    output_path = os.path.join(output_dir, file)
    cv2.imwrite(output_path, roi)
    print(f"Processed and saved {output_path}")


Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_0_0_20170112213500903_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_0_0_20170112215240346_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170110183726390_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170112213001988_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170112213303693_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170112215032192_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170117195420803_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_0_20170119212053665_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_2_20170105174847679_face0.jpg
Processed and saved C:\Users\DELL\Downloads\ikrima faces\part3\100_1_2_20170110182836729_face0.jpg
Processed 

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import os
import cv2
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

data_path = r"C:\Users\DELL\Downloads\ikrima faces\Cropped_face_dataset"

images = []
labels = []

for img_name in os.listdir(data_path):
    if img_name.endswith('.jpg'):
        img = cv2.imread(os.path.join(data_path, img_name))
        img = cv2.resize(img, (96, 96)) 
        images.append(img)
        
        parts = img_name.split('_')
        if len(parts) >= 2 and parts[1].isdigit():  
            label = int(parts[1])
            labels.append(label)
        else:
            print(f"Invalid file name format: {img_name}")

images = np.array(images)
labels = np.array(labels)

images = images / 255.0

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val))

model.save("gender_classification_model.h5")

val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation loss: {val_loss}')
print(f'Validation accuracy: {val_acc}')


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 94, 94, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,731,905 (6.61 MB)

 Trainable params: 1,731,905 (6.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 131s 253ms/step - accuracy: 0.6383 - loss: 0.6345 - val_accuracy: 0.7923 - val_loss: 0.4375
Epoch 2/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 101s 203ms/step - accuracy: 0.7870 - loss: 0.4535 - val_accuracy: 0.8052 - val_loss: 0.4298
Epoch 3/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 97s 196ms/step - accuracy: 0.8139 - loss: 0.4065 - val_accuracy: 0.8317 - val_loss: 0.3796
Epoch 4/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 90s 182ms/step - accuracy: 0.8221 - loss: 0.3899 - val_accuracy: 0.8423 - val_loss: 0.3550
Epoch 5/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 91s 184ms/step - accuracy: 0.8383 - loss: 0.3613 - val_accuracy: 0.8388 - val_loss: 0.3597
Epoch 6/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 91s 183ms/step - accuracy: 0.8444 - loss: 0.3513 - val_accuracy: 0.8580 - val_loss: 0.3323
Epoch 7/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 93s 188ms/step - accuracy: 0.8552 - loss: 0.3318 - val_accuracy: 0.8557 - val_loss: 0.3244
Epoch 8/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 94s 190ms/step - accuracy: 0.8657 - loss:

124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.8710 - loss: 0.3488
Validation loss: 0.3284769058227539
Validation accuracy: 0.8736735582351685


Here again we detect face crop the bottom 40 and try to predict 

In [13]:
test_image_path = r"C:\Users\DELL\Downloads\maskedfaces\labeled\0022_1.jpg"  
test_image = cv2.imread(test_image_path)
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# Load the pre-trained face detector (Haar cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
if len(faces) == 0:
            test_image = cv2.resize(test_image, (96, 96))  
            test_image = test_image / 255.0 
            input_image = np.expand_dims(test_image, axis=0)
else:
            (x, y, w, h) = faces[0]
            face_image = test_image[y:y+h, x:x+w]
            cropped_face_image = face_image[:int(0.60 * h), :]
            resized_image = cv2.resize(cropped_face_image, (96, 96))
            normalized_image = resized_image / 255.0
            input_image = np.expand_dims(normalized_image, axis=0)
predictions = model.predict(input_image)


if predictions[0][0] > 0.5:
       print(predictions[0][0])
       print("Predicted gender: Female")
else:
     print(predictions[0][0])
     print("Predicted gender: Male")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
0.00961801
Predicted gender: Male


Here I have labelled some test images now I try to predict them and calulate their accuracy

In [14]:
# Directory paths
test_images_dir = r"C:\Users\DELL\Downloads\maskedfaces\labeled"  # Directory with test images

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

correct_predictions = 0
total_images = 0

for file_name in os.listdir(test_images_dir):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        total_images += 1
        
        # Extract the true label from the filename
        true_label = int(file_name.split('_')[-1].split('.')[0])

        file_path = os.path.join(test_images_dir, file_name)
        test_image = cv2.imread(file_path)

        if test_image is None:
            print(f"Error loading image {file_path}")
            continue

        # Convert to grayscale for face detection
        gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(faces) == 0:
            print(f"No faces detected using Haar cascade in image {file_path}")
            test_image = cv2.resize(test_image, (96, 96)) 
            test_image = test_image / 255.0  
            input_image = np.expand_dims(test_image, axis=0)
        else:
            (x, y, w, h) = faces[0]
            face_image = test_image[y:y+h, x:x+w]
            cropped_face_image = face_image[:int(0.60 * h), :]
            resized_image = cv2.resize(cropped_face_image, (96, 96))
            normalized_image = resized_image / 255.0
            input_image = np.expand_dims(normalized_image, axis=0)
        predictions = model.predict(input_image)

        predicted_label = 1 if predictions[0][0] > 0.5 else 0

        print(f"Image: {file_name}, Predicted: {predicted_label}, True: {true_label}, Confidence: {predictions[0][0]}")

        if predicted_label == true_label:
            correct_predictions += 1

accuracy = np.mean(correct_predictions)
print(f"Accuracy: {accuracy }%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Image: 0002_1.jpg, Predicted: 1, True: 1, Confidence: 0.9173567295074463
No faces detected using Haar cascade in image C:\Users\DELL\Downloads\maskedfaces\labeled\0003_0.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Image: 0003_0.jpg, Predicted: 1, True: 0, Confidence: 0.5209214687347412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Image: 0004_0.jpg, Predicted: 1, True: 0, Confidence: 0.810391366481781
No faces detected using Haar cascade in image C:\Users\DELL\Downloads\maskedfaces\labeled\0005_0.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Image: 0005_0.jpg, Predicted: 0, True: 0, Confidence: 0.0009548950474709272
No faces detected using Haar cascade in image C:\Users\DELL\Downloads\maskedfaces\labeled\0006_0.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Image: 0006_0.jpg, Predicted: 0, True: 0, Confidence: 0.48274391889572144
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Image: 0007_1.jpg, Predicted: 1, True: 1, Confidence: 0.5351864099502563
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 